# Applied Data Science Capstone: Week 3 
## Segmenting and Clustering Neighborhoods in the city of Toronto, Canada, Part 3

A Jupyter Notebook that uses pandas and other python libraries to demonstrate 
*k means clustering*. Builds on work in Part 1 and Part 2 notebooks, which are
separate. New work begins under "Part 3".

## Code from Part 1

Like in the Part 2 notebook, I've condensed the code from part 1 into a single cell.

In [1]:
import pandas as pd
import lxml

df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
raw_postal_codes = df[0]
raw_postal_codes = raw_postal_codes[(raw_postal_codes.Borough != 'Not assigned')]
raw_postal_codes = raw_postal_codes[(raw_postal_codes.Neighbourhood != 'Not assigned')]

grouped = raw_postal_codes.groupby('Postal Code')
grouped_data = {'PostalCode':[], 'Borough':[], 'Neighborhood':[]}
for a, b in grouped:
    grouped_data['PostalCode'].append(a)
    grouped_data['Borough'].append(', '.join(b['Borough'].tolist()))
    grouped_data['Neighborhood'].append(', '.join(b['Neighbourhood'].tolist()))

postal_codes = pd.DataFrame(grouped_data)
print("Dataframe ready!")
print(postal_codes.shape)

Dataframe ready!
(103, 3)


## Code from Part 2

I've removed the geocoder and geopy code that didn't 
get results, and just load the CSV file.

In [2]:
long_lat = pd.read_csv("https://cocl.us/Geospatial_data")
long_lat.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
long_lat.head()
postal_codes = postal_codes.merge(long_lat, on='PostalCode')
print("Longitude and Latitude added to dataframe")
print(postal_codes.shape)

Longitude and Latitude added to dataframe
(103, 5)


## Part 3 code starts here

Let's explore the data, looking at similar things
in the New York exercise.

In [24]:
pip install folium

  Using cached folium-0.11.0-py2.py3-none-any.whl (93 kB)
  Using cached branca-0.4.1-py3-none-any.whl (24 kB)
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.2.7/libexec/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [25]:
from geopy.geocoders import Nominatim
import folium

In [26]:
postal_codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,NumberNeighborhoods
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,2
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,3
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1


### Determine number of Boroughs and Neighborhoods

I think it would be interesting to see the overall
number of boroughs and neighborhoods. However, since
the data has the Neighborhoods column is merged together,
the only way to get a count is to split and count the results

In [27]:
postal_codes['NumberNeighborhoods'] = postal_codes.apply(lambda row: len(row['Neighborhood'].split(",")), axis=1)

print('The dataframe has {} boroughs and {} neighborhoods'.format(
            len(postal_codes['Borough'].unique()),
            postal_codes['NumberNeighborhoods'].sum()))

The dataframe has 10 boroughs and 217 neighborhoods


### Use geopy to get the latitude and longitude values of Toronto, Ontario, Canada

In [28]:
address = "Toronto, Ontario, Canada"
geolocator = Nominatim(user_agent="Capstone_Week_3")
location = geolocator.geocode(address)
if location is not None:
    latitude = location.latitude
    longitude = location.longitude
    print("The coordinates of Toronto are {}, {}".format(latitude, longitude))
else:
    print("Coordinates not found!")

The coordinates of Toronto are 43.6534817, -79.3839347


### Create a map of Toronto with the postal codes superimposed on top

I'm also goig to color coordinate the points by borough

In [ ]:
print(postal_codes)

In [46]:
# color names come from the folium documentation

folium_colors = ['purple', 'black', 'green', 'red', 'orange', 'darkred',
                 'blue', 'beige', 'darkblue', 'darkgreen']

boroughs = postal_codes['Borough'].unique()
borough_colors = dict(zip(boroughs, folium_colors))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for i, row in postal_codes.iterrows():
    label_text = "{}, {}".format(row['PostalCode'], row['Borough'])
    label = folium.Popup(label_text, parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=5,
        popup=label,
        color=borough_colors[row['Borough']],
        fill=True,
        fill_color=borough_colors[row['Borough']],
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto